In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [2]:
sqlite3.sqlite_version

'3.35.4'

In [6]:
print('''
Есть таблица с данными по продажам кофе:
SALE_DTTM – точное время продажи,
CARD_NUMBER – номер скидочной карты
COFFEE_TYPE – название проданного кофейного напитка
PRICE – полная стоимость
DISCOUNT – размер скидки в %
GROCERY_FLG – флаг покупки выпечки в том же чеке

Составьте алгоритм SQL-запросов для решения следующих заданий:

1) Вычислить медианное значение суммарных месячных трат постоянных клиентов за каждый месяц
(при отсутствии встроенной функции, вычисляющей медиану).

2) Вычислить максимальное количество последовательных дней, когда один и тот же клиент покупал кофе с выпечкой.

3) В кофейне есть акция – при ежедневной покупке кофе с выпечкой можно купить термокружку со скидкой.
При этом размер скидки на термокружку получается путём последовательного применения всех скидок на кофе
с выпечкой, применённых за накопленную на момент покупки термокружки серию ежедневных покупок кофе с
выпечкой.Вычислить максимальную скидку на термокружку, которую мог достичь клиент (размер скидки клиента
не является константой).
''')


Есть таблица с данными по продажам кофе:
SALE_DTTM – точное время продажи,
CARD_NUMBER – номер скидочной карты
COFFEE_TYPE – название проданного кофейного напитка
PRICE – полная стоимость
DISCOUNT – размер скидки в %
GROCERY_FLG – флаг покупки выпечки в том же чеке

Составьте алгоритм SQL-запросов для решения следующих заданий:

1) Вычислить медианное значение суммарных месячных трат постоянных клиентов за каждый месяц
(при отсутствии встроенной функции, вычисляющей медиану).

2) Вычислить максимальное количество последовательных дней, когда один и тот же клиент покупал кофе с выпечкой.

3) В кофейне есть акция – при ежедневной покупке кофе с выпечкой можно купить термокружку со скидкой.
При этом размер скидки на термокружку получается путём последовательного применения всех скидок на кофе
с выпечкой, применённых за накопленную на момент покупки термокружки серию ежедневных покупок кофе с
выпечкой.Вычислить максимальную скидку на термокружку, которую мог достичь клиент (размер скидки 

In [21]:
df = pd.DataFrame({'dt': pd.date_range('2020-04-01', '2020-05-31')})
df['num'] = np.random.poisson(30, len(df))

In [22]:
df = pd.DataFrame({'sale_dttm':(df['dt'].apply(lambda x: [x])*df['num']).sum()})

In [50]:
df['card_number'] = pd.Series(list(range(40)) + [np.nan]*10).sample(len(df), replace=True).values

In [26]:
df['coffee_type'] = pd.Series(['espesso', 'americano', 'capuccino','capuccino','capuccino','latte']).sample(len(df), replace=True).values

In [31]:
coffee_price =  pd.DataFrame({'espesso': [100], 'americano': [120], 'capuccino': [130],'latte': [150]}).T.reset_index()

In [32]:
coffee_price.columns = ['coffee_type', 'coffee_price']

In [34]:
df = df.merge(coffee_price,how='left', on='coffee_type')

In [37]:
df['discount'] = pd.Series([0, 0, 0, 0, 0, 0.05, 0.05,0.05,0.05,0.1,0.15]).sample(len(df), replace=True).values

In [38]:
df['grocery_flg'] = pd.Series([0,0,0,1]).sample(len(df), replace=True).values

In [64]:
df['price'] = (df['coffee_price'] + df['grocery_flg']*100)*(1-df['discount'])

In [40]:
con = sqlite3.connect('data/db1')

In [87]:
cur = con.cursor()

In [65]:
df.to_sql('coffee_sales', con, index=False, if_exists='replace')

In [52]:
sql = '''select * from coffee_sales t limit 10'''

In [47]:
def select(sql):
    return pd.read_sql(sql, con)

In [53]:
select(sql)

,sale_dttm,card_number,coffee_type,coffee_price,discount,grocery_flg
0,2020-04-01 00:00:00,23.0,capuccino,130,0.15,0
1,2020-04-01 00:00:00,28.0,espesso,100,0.00,0
2,2020-04-01 00:00:00,NaN,capuccino,130,0.00,0
3,2020-04-01 00:00:00,NaN,espesso,100,0.05,0
4,2020-04-01 00:00:00,30.0,capuccino,130,0.05,0
5,2020-04-01 00:00:00,25.0,capuccino,130,0.00,1
6,2020-04-01 00:00:00,4.0,espesso,100,0.00,0
7,2020-04-01 00:00:00,22.0,espesso,100,0.05,0
8,2020-04-01 00:00:00,16.0,espesso,100,0.00,1
9,2020-04-01 00:00:00,37.0,capuccino,130,0.05,0


In [66]:
sql = '''
select date(t.sale_dttm, 'start of month') as month , t.card_number, sum(t.price) as price_sum
from coffee_sales t
group by  date(t.sale_dttm, 'start of month'), t.card_number
'''

select(sql)

,month,card_number,price_sum
0,2020-04-01,NaN,24965.5
1,2020-04-01,0.0,2196.5
2,2020-04-01,1.0,2833.5
3,2020-04-01,2.0,1633.0
4,2020-04-01,3.0,2111.5
...,...,...,...
77,2020-05-01,35.0,2538.0
78,2020-05-01,36.0,1454.0
79,2020-05-01,37.0,1299.5
80,2020-05-01,38.0,2271.5


In [75]:
df['sale_dttm'].values.astype('datetime64[M]')

array(['2020-04', '2020-04', '2020-04', ..., '2020-05', '2020-05',
       '2020-05'], dtype='datetime64[M]')

In [ ]:
df['month'] = df['sale_dttm'].values.astype('datetime64[M]')

In [81]:
df[:3]

,sale_dttm,card_number,coffee_type,coffee_price,discount,grocery_flg,price,month
0,2020-04-01,23.0,capuccino,130,0.15,0,110.5,2020-04-01
1,2020-04-01,28.0,espesso,100,0.00,0,100.0,2020-04-01
2,2020-04-01,NaN,capuccino,130,0.00,0,130.0,2020-04-01


In [78]:
m = df.groupby(['month', 'card_number'])['price'].sum().reset_index()
m

,month,card_number,price
0,2020-04-01,0.0,2196.5
1,2020-04-01,1.0,2833.5
2,2020-04-01,2.0,1633.0
3,2020-04-01,3.0,2111.5
4,2020-04-01,4.0,2754.5
...,...,...,...
75,2020-05-01,35.0,2538.0
76,2020-05-01,36.0,1454.0
77,2020-05-01,37.0,1299.5
78,2020-05-01,38.0,2271.5


In [80]:
m.groupby('month')['price'].agg(['count', 'median'])

,count,median
month,,
2020-04-01,40,2489.75
2020-05-01,40,2572.50


In [86]:
sql = '''
select x.month,
       avg(x.price_sum) as Median

from (select t.*,
             row_number() over (partition by t.month order by t.price_sum) as RowNum,
             count(*) over (partition by t.month)                          as RowCnt

      from (select date(t.sale_dttm, 'start of month') as month, t.card_number, sum(t.price) as price_sum
            from coffee_sales t
            group by date(t.sale_dttm, 'start of month'), t.card_number) t) x

where x.RowNum in ((x.RowCnt + 1) / 2, (x.RowCnt + 2) / 2)

group by x.month
'''

select(sql)

,month,Median
0,2020-04-01,2562.5
1,2020-05-01,2589.0


In [88]:
sql ='''
drop table IF EXISTS sales_day_1;
create table sales_day_1 as

select date(t.sale_dttm)                                           as dt,
       t.card_number,
       max(t.grocery_flg)                                             asgrocery_flg,
       count(1)                                                    as num_purchases,
       max(case when t.grocery_flg = 1 then t.discount else 0 end) as discount
from coffee_sales t
group by date(t.sale_dttm), t.card_number
;
'''

In [89]:
cur.executescript(sql)